In [ ]:
%load_ext autoreload
%autoreload 2

import struct
import datetime
import numpy as np
import os
import sys
import glob
import fnmatch

#sys.path.append('/Users/reckert/Documents/Develop/asdreader/')
import asdreader

%matplotlib widget
import matplotlib.pyplot as plt

### Basic read-in, reference ratio worflow

In [ ]:
base_dir = '/Users/reckert/Documents/Data/SHIFT_2022_Field/'
data_dir = base_dir + 'reginatest/'
file_tag = 'EPFL_VA_TC1'
filenameL = sorted([f for f in os.listdir(data_dir) if fnmatch.fnmatch(f, file_tag + '*.asd')])

In [ ]:
# Load in all asd_data
n_file = len(filenameL)

print('Reading files, beginning with {}...'.format(data_dir + filenameL[0]))
asd_data = asdreader.reader(data_dir + filenameL[0])

n_channels = asd_data.wavelengths.shape[0]
spectra_mat = np.zeros((n_file,n_channels))
for ii in range(len(filenameL)):
    spectra_mat[ii,] = asdreader.reader(data_dir + filenameL[ii],verbose=False).spec

In [ ]:
# View all measured spectra
plt.figure()
plt.imshow(spectra_mat,aspect='auto')
plt.colorbar()

In [ ]:
# Plot reference spectrum
known_ref_idx = 20
known_ref = spectra_mat[known_ref_idx,]

plt.figure()
plt.plot(asd_data.wavelengths,known_ref)

In [ ]:
def cos_sim_to_ref(x_mat,y_ref):
    return np.dot(x_mat,y_ref)/(np.sqrt(np.sum(x_mat**2,1))*np.sqrt(np.sum(y_ref**2)))

In [ ]:
#Identify reference spectra
# *** Could identify in another way, but here I am doing the simple thing of feeding in a known index
known_ref_idx = 20
known_ref = spectra_mat[known_ref_idx,]
cos_sim_vec = cos_sim_to_ref(spectra_mat,known_ref)

spectra_index = np.arange(spectra_mat.shape[0]) #*Could also load in datetime here from .asd files

ref_threshold = 0.999
ref_idx = cos_sim_vec>=ref_threshold
meas_idx = cos_sim_vec<ref_threshold

In [ ]:
#Find where reference spectra begin and end
ref_change = np.hstack([0,np.diff(ref_idx.astype(int))])
ref_begin_bnd = spectra_index[ref_change>0]
ref_end_bnd = spectra_index[ref_change<0]

#Case where we start with reference spectra
ref_begin_bnd = np.hstack([0, ref_begin_bnd]) if ref_begin_bnd[0]>ref_end_bnd[0] else ref_begin_bnd
#Case where we end with reference spectra
ref_end_bnd = np.hstack([ref_end_bnd, spectra_time[-1]]) if ref_begin_bnd[-1]>ref_end_bnd[-1] else ref_end_bnd

#*** Size of the begin and end bounds should be the same, but might want to check / deal with if they are not

In [ ]:
#Get average and standard deviation for reference groups
ref_mean = np.zeros((len(ref_begin_bnd),spectra_mat.shape[1]))
ref_std  = ref_mean.copy()

ref_groups = []
for ii in range(len(ref_begin_bnd)):
    ref_groups.append(spectra_mat[ref_begin_bnd[ii]:ref_end_bnd[ii],:])
    ref_mean[ii,:] = np.mean(spectra_mat[ref_begin_bnd[ii]:ref_end_bnd[ii],:],axis=0)
    ref_std[ii,:] = np.std(spectra_mat[ref_begin_bnd[ii]:ref_end_bnd[ii],:],axis=0)


In [ ]:
plt.figure(figsize=(8,4))
plt.subplot(1,2,1)
plt.plot(asd_data.wavelengths,np.transpose(ref_mean))
plt.title('Mean reference')
plt.subplot(1,2,2)
plt.plot(asd_data.wavelengths,np.transpose(ref_std))
plt.title('Standard deviation of reference')

In [ ]:
# *** Need to add in a way to declare outliers, especially on the reference groups
# *** Then update the ref_groups
# For now, setting a limit on standard deviation to do that
ref_good_idx = np.all(ref_std<1000,axis=1) 
ref_good_bnd = np.vstack((ref_begin_bnd[ref_good_idx],ref_end_bnd[ref_good_idx]))
ref_good_index = np.arange(len(ref_begin_bnd))[ref_good_idx] # Index into mean, std, and group arrays


In [ ]:
# *** Could do a much fancier interpolation scheme between the different reference groups
# For now, just identify which is closest and use that
ref_good_bnd_flat = ref_good_bnd.flatten()
ref_good_index_flat = np.vstack((ref_good_index,ref_good_index)).flatten()

#Find the closest reference group for each measurement
meas_ref_index = ref_good_index_flat[np.argmin(np.abs(np.expand_dims(spectra_index[meas_idx],axis=1) - np.expand_dims(ref_good_bnd_flat,axis=0)),axis=1)]
#Take the ratio
ratio_meas_ref = spectra_mat[meas_idx,:]/ref_mean[meas_ref_index,:]

In [ ]:
plt.figure()
plt.plot(asd_data.wavelengths,np.transpose(ratio_meas_ref))
plt.title('Ratio of target spectra to reference')
plt.ylabel('Reflectance')
plt.xlabel('Wavelength (nm)')

In [ ]:
plt.figure(figsize=(8,4))
plt.subplot(1,2,1)
plt.plot(asd_data.wavelengths,np.transpose(spectra_mat[meas_idx,:]))
plt.title('Raw target spectra')
plt.subplot(1,2,2)
plt.plot(asd_data.wavelengths,np.transpose(ref_mean[meas_ref_index,:]))
plt.title('Mean reference spectra')

In [ ]:
# *** Also need to be able to reject radiance outliers

### Reading in GPS data

In [ ]:
# **** GPS Data unpack needs work
# * Here is a place to test that out
header_format = ''
struct.unpack(header_format,asd_data.md.gps_data)

### Other verification plots

In [ ]:
#Verify the reference threshold is right
plt.figure()
plt.plot(cos_sim_vec)
plt.plot(ref_idx)

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
#Verify we are doing the cosine similarity calculation (i.e. normalized dot product) correctly
cos_sim_0 = cos_sim_to_ref(spectra_mat,known_ref)
known_ref_mat = np.tile(np.expand_dims(known_ref,axis=0),(spectra_mat.shape[0],1))
cos_sim_1 = cosine_similarity(spectra_mat,np.tile(np.expand_dims(known_ref,axis=0),(spectra_mat.shape[0],1)))[:,0]

plt.figure()
plt.plot(cos_sim_0)
plt.plot(cos_sim_1)